In [ ]:
!pip install azure-storage-blob
!pip install boto3
!pip install google-cloud-storage
!pip install sodapy

In [2]:
#import necessary libraries
import pandas as pd
import numpy as np
import json
import requests
import boto3
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.cloud import storage
from io import StringIO

s1 --> Gather --> Decompress --> Convert to Tabular --> Clean --> Reformat --> Consolidate --> Tranformation --> Load

In [9]:
from sodapy import Socrata
import pandas as pd
from sodapy import Socrata

data_url='data.cityofnewyork.us'    # The Host Name for the API endpoint (the https:// part will be added automatically)
data_set='vww9-qguh'    # The data set at the API endpoint (311 data in this case)
app_token='m93ORKHDVAEgZJ48OfuDNs5sI'   # The app token created in the prior steps
client = Socrata(data_url,app_token)      # Create the client to point to the API endpoint
# Set the timeout to 60 seconds
client.timeout = 60
# Retrieve the first 50000 res0ults returned as JSON object from the API
# The SoDaPy library converts this JSON object to a Python list of dictionaries
results = client.get(data_set, limit=50000, offset=1)
# Convert the list of dictionaries to a Pandas data frame
df = pd.DataFrame.from_records(results)
# Save the data frame to a CSV file

In [ ]:
df

In [ ]:
df.head()

In [4]:
# Function

import os
import boto3
from azure.storage.blob import BlobServiceClient
from google.cloud import storage
import pandas as pd
from io import BytesIO, StringIO

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [ ]:
from sodapy import Socrata
import pandas as pd
from sodapy import Socrata

data_url='data.cityofnewyork.us'    # The Host Name for the API endpoint (the https:// part will be added automatically)
data_set='vww9-qguh'    # The data set at the API endpoint (311 data in this case)
app_token='m93ORKHDVAEgZJ48OfuDNs5sI'   # The app token created in the prior steps
client = Socrata(data_url,app_token)      # Create the client to point to the API endpoint
# Set the timeout to 60 seconds
client.timeout = 60
# Retrieve the first 2000 results returned as JSON object from the API
# The SoDaPy library converts this JSON object to a Python list of dictionaries
results = client.get(data_set, limit=2000)
# Convert the list of dictionaries to a Pandas data frame
df_raw = pd.DataFrame.from_records(results)
df_raw.dropna()
df_raw.shape
df_raw.info()

In [6]:
df_clean = df_raw.copy()

#Chunking data from API into list

In [11]:
from sodapy import Socrata
import pandas as pd
data_url='data.cityofnewyork.us'    # The Host Name for the API endpoint (the https:// part will be added automatically)
data_set='vww9-qguh'    # The data set at the API endpoint (311 data in this case)
app_token='m93ORKHDVAEgZJ48OfuDNs5sI'   # The app token created in the prior steps
client = Socrata(data_url, app_token)
client.timeout = 60

# Define the chunk size
chunk_size = 50000
offset = 0
dfs = []

while True:
    # Retrieve data for the current chunk
    results = client.get(data_set, limit=chunk_size, offset=offset)

    # If there are no more results, break out of the loop
    if not results:
        break

    # Convert the chunk to a DataFrame and append it to the list of DataFrames
    df_chunk = pd.DataFrame.from_records(results)
    dfs.append(df_chunk)

    # Increment the offset for the next chunk
    offset += chunk_size



#Creating connection to Azure and uploading chunks into Azure Blob

In [15]:
import json
from io import StringIO
from azure.storage.blob import BlobServiceClient

# Assuming you already have the DataFrame chunks stored in 'dfs' list
#specify the path to your JSON configuration file
config_file_path = 'config.json'

with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

CONNECTION_STRING_AZURE_STORAGE = config['connectionString']
CONTAINER_AZURE = 'studentattendance'

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Iterate over DataFrame chunks and upload each chunk as a CSV file
for i, df_chunk in enumerate(dfs):
    # Convert DataFrame chunk to CSV string
    output = StringIO()
    df_chunk.to_csv(output, index=False)
    data = output.getvalue()
    output.close()

    # Define the blob name for the current chunk
    blob_name = f'nycatt_chunk_{i+1}.csv'

    # Get a blob client using the container name and blob name
    blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

    # Upload the CSV data
    blob_client.upload_blob(data, overwrite=True)

In [ ]:
# Example usage
if __name__ == "__main__":

    output = StringIO()
    df_raw.to_csv(output, index=False)
    data = output.getvalue()
    output.close()

    # Azure
    azure_connect_str = 'YOUR_AZURE_STORAGE_CONNECTION_STRING'
    azure_container_name = 'YOUR_CONTAINER_NAME'
    azure_blob_name = 'your_blob_name.csv'
    azure_data = 'Hello, Azure!'
    azure_upload_blob(azure_connect_str, azure_container_name, azure_blob_name, azure_data)
    # Download and print the blob content
    azure_blob_content = azure_download_blob(azure_connect_str, azure_container_name, azure_blob_name)
    print(azure_blob_content)

In [9]:
#specify the path to your JSON configuration file
config_file_path = 'config.json'

#load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

#print the configuration
#Connection_STRING = config['CONNECTION_STRING_AZURE_STORAGE']

CONNECTION_STRING_AZURE_STORAGE = config['connectionString']
CONTAINER_AZURE = 'studentattendance'
blob_name = 'nycatt.csv'

# Convert DataFrame to CSV
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded nycatt.csv to Azure Blob Storage in container studentattendance.
